#  High-thoughput ab initio calculation with Python

Tutorial para o evento **Machine Learning School for Materials**, Ilum, CNPEM, 2022.

Henrique Ferreira dos Santos (hfsantos@ufabc.edu.br)

---------------------------------

# Parte 1 - Screening 

Antes de iniciar as contas **High Throughput** vamos obter informações de bases de dados de contas DFT.

O **screening** é o procedimento de busca e seleção de materiais nessas bases de dados, de maneira automatizada e baseado em critérios estebelecidos pelo pesquisador.

Como exemplo, vamos realizar um estudo de semicontudores de **Ultrawide Bandgap** (UWBG semiconductors). Inicialmente vamos buscar nas bases de dados compostos químicos que já foram simulados e apresentam gap ultra largo. Em seguida, vamos filtrá-los com uma série de critérios que iremos estabelecer. Por fim, selecionaremos um composto da lista final para realizar um estudo mais detalhado que será feito na Parte 2 desse tutorial.


### Bibliotecas utilizadas

In [ ]:
#!pip install pymatgen  # Caso não tenha a biblioteca pymatgen, você pode instalá-la usando este comando

In [ ]:
# Pymatgen is a open-source librarie for materials analysis
from pymatgen.ext.matproj  import MPRester                         # API requester for Materials project
from pymatgen.core.periodic_table import Element                   # Class to represent Element in pymatgen
from pymatgen.io import vasp                                       # Interface with VASP
from pymatgen.electronic_structure.plotter import BSPlotter        # Plotar a estrutura de bandas

import pandas as pd   # Working with tables

import os
import time

## Critérios de busca

- [ ] Materiais ternários
- [ ] Elementos não radioativos
- [ ] Bandgap entre 4 e 12 eV
- [ ] Têm gap direto
- [ ] Estrutura de bandas foi reportada
- [ ] Tem entrada no ICSD
- [ ] Materiais termodinamicamente estáveis (estão no convex hull)
- [ ] Tem pelo menos uma rota de síntese conhecida

### Elementos não radioativos

In [ ]:
# Gerando a lista de elementos não radioativos

def desired_element(elem):
    omit = ['Po', 'At', 'Rn', 'Fr', 'Ra']
    return not elem.is_noble_gas and not elem.is_actinoid and not elem.symbol in omit

element_universe = [e for e in Element if desired_element(e)]
omitted_elements = [e for e in Element if e not in element_universe]
elements = [e.symbol for e in element_universe] 

print("Número de elementos incluídos =", len(element_universe))
print("Elementos excluídos: ", " ".join(sorted([e.symbol for e in omitted_elements])))

### Base de busca - Materials Project
O Materials Project é um base de dados pública. 

Os critérios de busca do Materials Project são limitados a uma lista de propriedades que pode ser consultada aqui https://workshop.materialsproject.org/lessons/04_materials_api/MAPI%20Lesson%20%28filled%29/.

O Materials Project é feito usando tecnologia MongoDB para o banco de dados. Dessa forma, a query pode ser aperfeiçoada usando a sintaxe desse banco de dados. A sintaxe pode ser consultada em https://www.mongodb.com/docs/manual/reference/operator/query/

As propriedades que são possiveis de serem capturadas estão em https://github.com/materialsproject/mapidoc/materials. A princípio, todas as pastas são informações que podem ser acessadas. Entretanto, nem todas as informações podem ser acessadas em uma query geral (precisam ser acessadas via query específica para um material) e algumas delas podem não estar presentes (faltam para aquele material).

In [ ]:
# Critérios de busca na base de dados
criteria = {'nelements':{'$in': [3]},            # Somente materiais ternários
            'band_gap':{'$gte': 4, '$lte': 12},  # Bandgap está entre 4 e 12 (gte >= and lte <=)
            'elements':{'$in':elements},         # Lista de elementos permitidos
            '$where':'this.icsd_ids.length>0',   # Tem entrada no ICSD
            'band_gap.search_gap.is_direct': {'$eq': True}, # Bandgap direto
            'has_bandstructure':{'$eq': True}    # Estrutura de bandas foi calculada
           } 

# Propriedades buscadas
properties =['material_id', 'icsd_ids', 'pretty_formula','elements', 'band_gap','formation_energy_per_atom',
             'e_above_hull', 'spacegroup']

# Chave de acesso
apikey = ''

In [ ]:
# Chamada API rest para requisitar os dados ao servidor do MP
with MPRester(apikey) as mpr:
    results = mpr.query(criteria, properties)

Armazenamos os resultados da requisição (488 materiais) na variável <code>result</code>.

Para facilitar o trabalho com essa variável, vamos transformá-la em uma tabela via pandas do tipo <code>DataFrame</code>:

In [ ]:
mat_list0 = pd.DataFrame(data = results)

In [ ]:
type(mat_list0)

In [ ]:
len(mat_list0) # Quantidade de entradas retornadas

In [ ]:
mat_list0.head(10) # Olhando as 10 primeiras linhas da tabela de materiais

A coluna <code>spacegroup</code> é do tipo dicionário (dict). Vamos transformar cada chave do dicionário em uma nova coluna:

In [ ]:
# Rearranja as informações dos grupos espaciais
mat_list0[['symprec',
          'source',
          'symbol',
          'number',
          'point_group',
          'crystal_system',
          'hall']] = mat_list0.spacegroup.apply(pd.Series)
mat_list0 = mat_list0.drop('spacegroup', axis=1)

In [ ]:
mat_list0.head()

### Estatísticas Básicas

Pelos nossos filtros de busca, a solicitação para o banco de dados retornou apenas materiais que respeitavam os seguintes critérios:

- [x] Materiais ternários
- [x] Elementos não radioativos
- [x] Bandgap entre 4 e 12 eV
- [x] Têm gap direto
- [x] Estrutura de bandas foi reportada
- [x] Tem entrada no ICSD
- [ ] Materiais termodinamicamente estáveis (estão no convex hull)
- [ ] Tem pelo menos uma rota de síntese conhecida
Vamos visualizar algumas informações estatísticas básicas dos compostos que temos até o momento:

In [ ]:
mat_list0.describe() # Gera informações estatísticas das colunas numéricas

In [ ]:
mat_list0.hist('band_gap')  # Plota o histograma do bandgap

In [ ]:
mat_list0.crystal_system.value_counts().plot(kind='bar')  # Distribuição dos sistemas cristalinos

In [ ]:
mat_list0.groupby('crystal_system')['band_gap'].describe()

### Materiais Estáveis
Agora vamos filtrar apenas os estáveis:

In [ ]:
mat_list1 = mat_list0[mat_list0['e_above_hull']==0]

In [ ]:
mat_list1.describe()

### Rotas de Síntese

Vamos verificar se existem rotas de sintese conhecidas para esses materiais. Para fazer isso precisaremos de dados adicionais que não existem no Materials Project. Aqui vamos utilizar a base de dados disponibilizada em https://github.com/CederGroupHub/text-mined-synthesis_public. Essa base de dados foi levantada usando-se ferramentas de Processamento de Linguagem Natural em cima de diversos artigos científicos.

No link, existem três arquivos <code>.json</code> comprimidos no formato <code>.xz</code>. Vamos escolher o  <code>solid-state_dataset_2019-12-03.json.xz</code>. Devemos baixá-lo e descompactá-lo. Em seguida, caso o ambiente usado seja o Google Colab, você deverá subir o arquivo na pasta lateral esquerda, ou usar o seguinte comando para baixar o arquivo e descompactá-lo automaticamente (atenção, este comando funciona melhor em ambientes Linux, como o Google Colab - se você estiver rodando na sua máquina local com Windows, considere abrir o arquivo diretamente pulado a próxima célula):

In [ ]:
#!wget https://github.com/CederGroupHub/text-mined-synthesis_public/raw/master/solid-state_dataset_2019-12-03.json.xz && xz -d solid-state_dataset_2019-12-03.json.xz

In [ ]:
synth_data = pd.read_json('solid-state_dataset_2019-12-03.json')

In [ ]:
synth_data.head()

In [ ]:
len(synth_data) # Quantidade de entradas

Como as informações que vieram estão em duas colunas, vamos olhar quais são os atributos do objeto <code>dict</code> que estão na coluna <code>reactions</code>.

In [ ]:
synth_data['reactions'][0].keys()

Da mesma forma que fizemos para o grupo espacial, podemos fazer para este caso, criando uma nova coluna para cada atributo (chave) do dicionário:

In [ ]:
# Rearranja as informações 
synth_data[list(synth_data['reactions'][0].keys())] = synth_data.reactions.apply(pd.Series)
synth_data = synth_data.drop('reactions', axis=1) # Deletando a coluna reactions original

In [ ]:
synth_data.head()

Como queremos saber se o composto alvo da síntese está na nossa lista de compostos UWBG iremos arrumar a coluna <code>target</code>: 

In [ ]:
synth_data['target'][0].keys()

In [ ]:
# Rearranja as informações 
synth_data[list(synth_data['target'][0].keys())] = synth_data.target.apply(pd.Series)
synth_data = synth_data.drop('target', axis=1)

In [ ]:
synth_data.head()

Agora vamos usar a coluna <code>mp_id</code> que contém um identificador de entrada no Materials Project para comparar os dois:

In [ ]:
mat_list2 = mat_list1.merge(synth_data, left_on='material_id', right_on='mp_id')

In [ ]:
mat_list2.head()

Neste ponto podemos ter mais de uma linha para o mesmo composto caso haja mais de um rota de síntese na base de dados usada. Observe que na tabela dos compostos do MP haviamos encotnrado apenas 249, e que acgoram temos 340 entradas (mais de uma por material!). Podemos querer uma versão reduzida somente com os compostos únicos:

In [ ]:
len(mat_list2) # Total de linhas (podendo conter duplicidades devido a mais de uma rota de síntese)

In [ ]:
mat_list3 = mat_list2.drop_duplicates(subset=['material_id'])

In [ ]:
len(mat_list3) # Lista reduzida com compostos únicos

Dessa forma, concluímos que apenas 38 materiais atendem os critérios estabelecidos, levando-se em consideração as duas bases de dados utilizadas:

- [x] Materiais ternários
- [x] Elementos não radioativos
- [x] Bandgap entre 4 e 12 eV
- [x] Têm gap direto
- [x] Estrutura de bandas foi reportada
- [x] Tem entrada no ICSD
- [x] Materiais termodinamicamente estáveis (estão no convex hull)
- [x] Tem pelo menos uma rota de síntese conhecida

In [ ]:
mat_list3['pretty_formula'].values

Vamos escolher um desses materiais para estudar em detalhe: **CaAlF5**

In [ ]:
selected_material = mat_list2[mat_list2['pretty_formula']=='CaAlF5'] # Estamos pegando na lista 2 com todas as rotas

In [ ]:
selected_material

In [ ]:
selected = 'mp-8836' # ID do Materials Project do material escolhido

In [ ]:
selected_material.keys() # Propriedades nas tabelas manipuladas

### Rota de Síntese

In [ ]:
selected_material['reaction_string'].values

In [ ]:
selected_material['operations'].values

### Estrutura Eletrônica

In [ ]:
with MPRester(apikey) as mpr:
    bs = mpr.get_bandstructure_by_material_id(selected)

In [ ]:
type(bs)

In [ ]:
bs.get_band_gap()

In [ ]:
efermi=bs.efermi
print(efermi)  # Energia de Fermi eV

In [ ]:
bsp = BSPlotter(bs)
bsp.get_plot(zero_to_efermi=True).show() # Plote automatico do Pymatgen
#bsp.bs_plot_data(zero_to_efermi=True)   # Pega os dados em forma de dicionário para fazer o plot manual

### Estrutura Cristalina

In [ ]:
with MPRester(apikey) as mpr:
    structure = mpr.get_structure_by_material_id(selected)

In [ ]:
structure.lattice.alpha

In [ ]:
# OBS: A variável bs que armazena a estrutura eletrônica, também armazena a estrutura cristalina
#bs.structure

Vamos salvar essa estrutura em um arquivo <code>POSCAR</code> dentro de uma pasta com o nome do composto <code>proto_CaAlF5</code>, que ficará dentro de uma pasta geral <code>ht</code>. 

In [ ]:
poscar = vasp.inputs.Poscar(structure)

In [ ]:
master_folder = 'ht'
material_folder = 'proto_CaAlF5'
filename = 'POSCAR'

In [ ]:
os.mkdir(master_folder)
os.mkdir(master_folder+'/'+material_folder)

In [ ]:
poscar.write_file(master_folder+'/'+material_folder+'/'+filename)

Uma pergunta interessante é: <font color='red'>se trocarmos cada elemento desse composto por outro elemento com alta semelhança química, ainda obtemos um UWBG estável e sintetizável?</font> 

Visando responder isso, vamos realizar a Parte 2 desse tutorial, onde definiremos simulações DFT de novos materiais.

-------------------------------------------------

# Parte 2 - Configurando as entradas para contas HT 

Nesta etapa, já temos a pergunta e o objeto de estudo oriundas da Parte 1: CaAlF5 (mp-8836)

Vamos estabelecer que a similaridade química será definida por elementos do mesmo grupo químico. Vamos desconsiderar o Rádio (Ra) e o Ástato (At).

Como *solver* de Mecânica Quântica, vamos utilizar o software VASP.

Estamos assumindo que as contas serão realizadas em um cluster com gerenciador de fila (como o SLURM) e que não iremos utilizar nenhuma biblioteca adicional para auxiliar no gerenciamento das contas, exceto aquelas de pré-processamento (Parte 2) e pós-processamento (Parte 3).

Em caso de uso de clusters sem fila, sugere-se uso da biblioteca ASE (https://wiki.fysik.dtu.dk/ase/) para submissão de cálculos direto pelo código Python. Para usuários mais avançados que requeiram altas vazões de contas, inclusive em clusters com gereciamento de fila, sugere-se o uso do AiiDA (https://www.aiida.net/).

In [ ]:
from pymatgen.io import vasp                                       # Interface with VASP
from pymatgen.core.structure import Structure                      # Class to represent Structures in pymatgen
from pymatgen.core.composition import Composition                  # Class to represent Composition in pymatgen
from pymatgen.core.periodic_table import Element                   # Class to represent Element in pymatgen
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer          # Methods to analyze space groups of materials

import shutil
import os

### Criando novos materiais e configurando arquivos VASP

Nessa primeira etapa estamos interessados em fazer a atualização das posições ionicas para encontrar as estruturas com menor enegia (convergidas).

Vamos utilizar a estrutura original do CaAlF5 como protótipo e manter a simetria, trocando apenas os átomos dos elementos da mesma coluna da tabela periódica.

Primeiro vamos carregar nosso protótipo:

In [ ]:
input_folder = 'ht/proto_CaAlF5'  # Pasta onde está a estrutura inicial (POSCAR) usada como protótipo
proto_poscar = Structure.from_file(input_folder+"/POSCAR")  # Estrutura protótipo

In [ ]:
proto_space_group = SpacegroupAnalyzer(proto_poscar, symprec=0.1, angle_tolerance=1.0)
proto_space_group.get_space_group_number()

Vamos usar as seguintes configurações de arquivos VASP:

In [ ]:
incar_rx = {
"ALGO": "Normal",
"EDIFF": "0.000001",
"ENCUT": 520,
"IBRION": 2,
"ISIF": 3,
"ISMEAR": 0,
"ISPIN": 1,
"LASPH": True,
"LORBIT": 11,
"LREAL": "Auto",
"LWAVE": False,
"LCHARG":False,
"NELM": 100,
"NSW": 900,
"SIGMA": 0.05,
"NPAR": 6
}

auto_kpoints = 25

Agora iremos criar a estrutura de pasta com todos os arquivos necessários para submeter as contas e todos os novos compostos:

In [ ]:
# Lista de elementos que serão usados para fazer as substituições químicas
earthalk_list = ['Be','Mg','Ca','Sr','Ba']
boron_list = ['B','Al','Ga','In','Tl']
halogen_list = ['F','Cl','Br','I']

In [ ]:
calculations_folder = 'ht/rx'
os.mkdir(calculations_folder)
pseudos_src_path = 'pp/'

for e1 in earthalk_list:
    for e2 in boron_list:
        for e3 in halogen_list:
            
            # Nova pasta para o novo material em disco
            output_folder=calculations_folder+'/'+e1+e2+e3+'5'
            os.mkdir(output_folder)
            
            temp_structure = proto_poscar.copy()          # Cópia da estrutura original em memória

            if not(e1=='Ca' and e2=='Al' and e3=='F'):
                for i, element in enumerate(proto_poscar):    # Para dada atomo na estrutura
                    element_str = str(element.species)        # Pega o atomo no sítio
                    element_str = element_str.replace('1','') # Remove o número de estequiometria (no caso o número é 1)

                    # Troca apenas elementos da mesma familia
                    if element_str in earthalk_list:
                        temp_structure.replace(i,e1)
                    elif element_str in boron_list:
                        temp_structure.replace(i,e2)
                    elif element_str in halogen_list:
                        temp_structure.replace(i,e3)

            # Aviso de que a eventualmente uma estrutura não manteve a simetria pretendida
            space_group = SpacegroupAnalyzer(temp_structure, symprec=0.1, angle_tolerance=1.0)
            if not(space_group.get_space_group_number()==proto_space_group.get_space_group_number()):
                print('Warning: Estrutura '+e1+e2+e3+'5 com grupo espacial diferente do protótipo!') 

            # Criar o novo POSCAR na pasta de destino
            new_poscar = vasp.inputs.Poscar(temp_structure)
            new_poscar.write_file(output_folder+'/POSCAR')
            
            # Criar o arquivo KPOINTS na pasta de destino
            kpoints = vasp.Kpoints().automatic(auto_kpoints)  # Método automatico com auto_kpoints (25) pontos
            kpoints.write_file(output_folder+'/KPOINTS')
            
            # Criar arquivo INCAR na pasta de destino
            incar = vasp.Incar(incar_rx)
            incar.write_file(output_folder+'/INCAR')
            
            # Criar arquivo POTCAR na pasta de destino
            
            
            # Criar arquivo JOB na pasta de destino
            

Aqui são fornecidos dois exemplos de funções que podem ser usadas para criar arquivos <code>POTCAR</code> e <code>JOB</code>.

<pre><code>
# INPUT: recebe o caminho da pasta de pseudos (passar o caminho inteiro para o potencial PBE, GGA, etc)
# OUTPUT: cria um arquivo POTCAR e retorna uma flag de sucesso (True) ou falha (False)
def create_potcar(compound, pseudos_src_path,  best_choice, dst_path):

    # Cria a pasta para colocar os POTCAR caso ela não exista
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    comp1 = Composition(compound)
    elements=[]
    final_file = []
    for elem in comp1.formula.split(" "):
        n = re.sub('\D', '', elem)
        element = elem.replace(n,'')

        if element in best_choice.keys():
            element_path = pseudos_src_path+'/'+element+best_choice[element]+'/POTCAR'
            #print(element_path)
            # Se não existir a melhor escolha para aquela pasta de PSEUDOS
            if not(os.path.isfile(element_path)):
                element_path = pseudos_src_path+'/'+element+'/POTCAR'
        else:
             element_path = pseudos_src_path+'/'+element+'/POTCAR'

        #print(element_path)

        # Se o arquivo de POTCAR existir
        if os.path.isfile(element_path):
            with open(element_path, 'r') as file:
                x = file.readlines()
            final_file = final_file+x
            response=True
        else:
            print("Not found POTCAR of: "+element)
            response=False
            break

    if response and len(final_file)>0:
        complete_dst_path = dst_path+'/POTCAR'
        with open(complete_dst_path, 'w') as file:
            file.writelines(final_file)

    return response


# INPUT: caminho do prototipo do job, nome do job, tempo estimado de uso, caminho de destino do arquivo
# OUTPUT: criar arquivo job na pasta de destino e retorna uma flag de sucesso (True) ou falha (False)
def create_jobfile(protojob_path, job_name, time, job_dst_path):

    if os.path.isfile(protojob_path):
        # Cria a pasta para colocar o arquivo JOB caso ela não exista
        if not os.path.exists(job_dst_path):
            os.mkdir(job_dst_path)

        # Carrega o arquivo prototipo do job
        with open(protojob_path, 'r') as file:
            job = file.readlines()

        new_job=[]
        for line in job:
            line = line.replace('jobname',job_name)
            line = line.replace('xx:xx:xx',time)
            new_job.append(line)

        complete_path = job_dst_path+'/'+job_name
        #complete_path = job_dst_path+'/JOB'

        with open(complete_path,'w') as file:
            file.writelines(new_job)

        response=True

    else:
        print('Job proto file not found')
        response = False

    return response
</code></pre>

Uma vez que estamos no ambiente Google Colab, podemos compactar nossos arquivos criados para baixá-los e usá-los em outro ambiente computacional:

In [ ]:
#!zip -r '/content/ht_todo.zip' '/content/ht/'

# Parte 3 - Submentendo as contas para o solver de MQ

A submissão de contas para o solver de Mecânica Quântica é fortemente atrelado ao ambiente computacional que será usado.

Por exemplo, se a conta será submetida em um computador pessoal desktop com o software instalado, ou um cluster de computação científica com vários nós de processamento. Cada ambiente pede uma forma de submissão diferente.

Abaixo mostramos a ideia básica de submissão usando Python puro, contudo existem outras bibliotecas que podem ser usadas como o ASE e o AiiDA. É aconselhável separar o código abaixo para um script dedicado <code>.py</code>, que pode ser carregado em um cluster.

Não iremos executar as funções propriamente ditas (apenas um <code>print</code> será visto) uma vez que o Google Colab não tem um software de Mecânica Quântica instalado para realizar as simulações físicas.

<pre><code>
import os
import shutil
import time
import subprocess
import argparse

# Retorna o numero de espacos livres na fila do SLURM
def check_free_spot(user):

    temp = 'check_free_spot.txt'
    script = f'''squeue -u {user} > {temp}'''

    count_lines = subprocess.Popen(script, stdout=subprocess.PIPE, shell=True)
    time.sleep(2)
    with open(temp, 'r') as file:
        lines = file.readlines()

    n_lines = 21-len(lines)

    os.remove(temp)

    return n_lines

# Submete o comando do script
def submit_job(job_path):

    job = job_path.split('/')[-1]
    shell_command='sbatch '+job
    job_path=job_path.replace(job,'')

    submit = subprocess.Popen(shell_command,
                              stdout=subprocess.PIPE,
                              shell=True,
                              cwd=job_path)
    status = submit.communicate()[0].decode('UTF-8')

    if status.split()[0] == 'Submitted':
        print('Submission successful.')
        response=True
    else:
        print('Submission failed.')
        response=False

    return(response)


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-jb', '--jobs', default = 'jobs_submetidos.txt',
                        help = 'Nome do arquivo de jobs submetidos.', type = str)

    parser.add_argument('-fjb', '--fjobs', default = 'jobs_afazer',
                        help = 'Pasta com arquivos de jobs a serem submetidos.', type = str)

    parser.add_argument('-u', '--user', default = 'usuario_da_sd',
                        help = 'Usuario do SLURM.', type = str)

    args = parser.parse_args()
    # Verifica se existe um arquivo com os jobs ja nfeitos
    if not(os.path.isfile(args.jobs)):
        f= open(args.jobs,"w+")
        f.close()
        nfeitos=0
    else:
        with open(args.jobs, 'r') as file:
            jobs_prontos = file.readlines()
        nfeitos=len(jobs_prontos)


    sleep_time = 1800
    qts_jobs = len(os.listdir(args.fjobs))
    todo_jobs = qts_jobs-nfeitos # qtd de jobs que ainda precisam ser feitos
    i = 0


    while(i<todo_jobs):

        with open(args.jobs, 'r') as file:
            jobs_prontos = file.readlines()

        jobs_prontos=[job.replace('\n','') for job in jobs_prontos] # So tira o \n

        for folder in os.listdir(args.fjobs):
            # O job ainda nao foi submetido
            if not(folder in jobs_prontos):
                n_lines = check_free_spot(args.user)
                # Tem espaco para colocar na fila
                if n_lines>0:
                    job_folder = args.fjobs+'/'+folder
                    job_file = [file for file in os.listdir(job_folder) if 'job' in file][0]
                    submit_job(job_folder+'/'+job_file)
                    with open(args.jobs, 'a') as file:
                        file.write(folder+'\n')
                    i = i+1
                else:
                    break
        time.sleep(sleep_time)

if __name__ == "__main__":
    main()
</code></pre>

# Parte 4 - Analisando resultados

Neste tutorial vamos analisar os resultados prontos após algumas etapas de contas, além da relaxação iônica. Metodologias DFT mais robustas pedem por uma etapa de otimização sem movimento iônico e um novo cálculo com os caminhos de alta simetria para obter o diagrama de bandas eletrônicas. Neste caso, vamos deixar mais simples sem perda de generalidade: tudo aqui mostrado pode ser adaptado para outras metodologias mudando os parâmetros do INCAR e KPOINTS para o caso do Vasp, inclusive repetindo a Etapa 3 conforme o necessário. Outros solvers de mecânica quântica podem ser usados com integração com o Pymatgen (veja https://pymatgen.org/pymatgen.io.html para informações sobre as interfaces com outros softwares de mecânica quântica como Lammps).


Os resultados parciais aqui mostrados fazem parte de pesquisa em andamento, cuja última referência é:

- H. Ferreira, J. A. Souza, and G. M. Dalpian, **High-throughput Calculations to Discovery New Compounds: the case of Jakobssonite**, Encontro de Outono da Sociedade Brasileira de Física, 2022.


### Carregando os dados

O VASP gera vários arquivos de resultados, que podem ser carregados e examinados. Aqui estamos trabalhando apenas com um recorte desses arquivos <code>CONTCAR</code> (para estrutura cristalina) e <code>EIGENVAL</code> (para bandgap).

In [ ]:
# Copiando as informações do repositório do GITHUB
#!wget https://github.com/simcomat/matinfo_tutorials/tree/main/ilum_2022/results_bs.rar

# Descompactando as informações no computador do Google Colab
#!unrar x results_bs.rar

In [ ]:
calculations_results = 'results_bs'
results = []
for e1 in earthalk_list:
    for e2 in boron_list:
        for e3 in halogen_list:
            
            folder=f'{calculations_results}/{e1}{e2}{e3}5'
            
            contcar = Structure.from_file(f'{folder}/CONTCAR')
            eigenval = vasp.outputs.Eigenval(f'{folder}/EIGENVAL')
            
            band = eigenval.eigenvalue_band_properties 
            space_group = SpacegroupAnalyzer(contcar, symprec=0.1, angle_tolerance=1.0)
            
            proprieties = {
                'material':f'{e1}{e2}{e3}5',
                'A':e1,
                'B':e2,
                'C':e3,
                'bandgap':band[0],
                'cbm':band[1],
                'vbm':band[2],
                'bandgap_direct':band[3],
                'lattice_a':contcar.lattice.a,
                'lattice_b':contcar.lattice.b,
                'lattice_c':contcar.lattice.c,
                'lattice_alpha':contcar.lattice.alpha,
                'lattice_beta':contcar.lattice.beta,
                'lattice_gamma':contcar.lattice.gamma,
                'space_group':space_group.get_space_group_number()
            }
            results.append(proprieties)
            

In [ ]:
results_df = pd.DataFrame(data=results)  # Colocando em uma tabela

In [ ]:
results_df.head()

As estatísticas básicas podem ser obtidas usando o comando <code>.describe</code> sobre a variável <code>DataFrame</code> que armazena os dados de resultados carregados:

In [ ]:
results_df.describe()

### Gerando gráficos
Para gerar gráficos melhores podemos usar as bibliotecas <code>matplotlib</code> e <code>seaborn</code>:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,4), dpi=100)

sns.scatterplot(data=results_df, x="lattice_a", y="bandgap", hue='bandgap_direct', ax=ax[0])
sns.scatterplot(data=results_df, x="lattice_b", y="bandgap", hue='bandgap_direct', ax=ax[1])
sns.scatterplot(data=results_df, x="lattice_c", y="bandgap", hue='bandgap_direct', ax=ax[2])

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,4), dpi=100)

sns.scatterplot(data=results_df, x="lattice_alpha", y="bandgap", hue='bandgap_direct', ax=ax[0])
sns.scatterplot(data=results_df, x="lattice_beta", y="bandgap", hue='bandgap_direct', ax=ax[1])
sns.scatterplot(data=results_df, x="lattice_gamma", y="bandgap", hue='bandgap_direct', ax=ax[2])

Figuras mais elaboradas podem ser feitas usandos os parâmetros dos métodos das bibliotecas de plot (<code>matplotlib</code> e <code>seaborn</code> - também é possível usar <code>latex</code> caso haja uma instalação local do mesmo e um módulo de compatibilidade instalado).

Abaixo mostramos alguns exemplos:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
#### - --------

sns.boxplot(x="A", y="bandgap",
            palette="hls",
            linewidth=0.7,
            ax=axes[0],
            data=results_df,
            order=['Be','Mg','Ca','Sr', 'Ba'],
            #cut=0
           )

sns.boxplot(x="B", y="bandgap",
            palette="hls",
            linewidth=0.7,
            ax=axes[1],
            data=results_df,
            order=['B','Al','Ga','In', 'Tl'],
            #cut=0
           )

sns.boxplot(x="C", y="bandgap",
            palette="hls",
            linewidth=0.7,
            ax=axes[2],
            data=results_df,
            order=['F','Cl','Br','I'],
            #cut=0
           )


### Ajustes
axes[0].set_xlabel('Alkaline Earth Metal',  fontsize=14)
axes[1].set_xlabel('Boron Group',  fontsize=14)
axes[2].set_xlabel('Halogen',  fontsize=14)

axes[0].set_ylim([-1,8])
axes[1].set_ylim([-1,8])
axes[2].set_ylim([-1,8])

axes[0].set_ylabel('Bandgap Energy (eV)',  fontsize=13)
axes[1].set_ylabel('Bandgap Energy (eV)',  fontsize=13)
axes[2].set_ylabel('Bandgap Energy (eV)',  fontsize=13)


# We change the fontsize of minor ticks label 
axes[0].tick_params(axis='both', which='major', labelsize=16)
axes[0].tick_params(axis='both', which='minor', labelsize=16)
axes[1].tick_params(axis='both', which='major', labelsize=16)
axes[1].tick_params(axis='both', which='minor', labelsize=16)
axes[2].tick_params(axis='both', which='major', labelsize=16)
axes[2].tick_params(axis='both', which='minor', labelsize=16)

plt.subplots_adjust(wspace=0.25)
#plt.savefig('bandgap_boxplot.png', transparent=True) Para salvar a imagem

In [ ]:
dados2 = results_df.copy()

dados2['alkaline_earth'] =  pd.Categorical(dados2['A'], ['Be','Mg','Ca','Sr', 'Ba'])
dados2['boron_family'] =  pd.Categorical(dados2['B'], ['B','Al','Ga','In', 'Tl'])
dados2['halogen'] =  pd.Categorical(dados2['C'], ['F','Cl','Br','I'])

fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)

sns.histplot(x="alkaline_earth",hue="bandgap_direct",
            palette=["grey", "yellow"],
            linewidth=0.7, ax=axes[0],
            data=dados2, multiple="dodge", shrink=.8,
            legend=None
           )

sns.histplot(x="boron_family", hue="bandgap_direct",
            palette=["grey", "yellow"],
            linewidth=0.7, ax=axes[1],
            data=dados2, multiple="dodge", shrink=.8,
            legend=None
           )

sns.histplot(x="halogen", hue="bandgap_direct",
            palette=["grey", "yellow"],
            linewidth=0.7, ax=axes[2],
            data=dados2, multiple="dodge", shrink=.8,
            legend=None
           )

axes[0].grid(axis='y', alpha=0.5)
axes[1].grid(axis='y', alpha=0.5)
axes[2].grid(axis='y', alpha=0.5)

axes[0].set_xlabel('Alkaline Earth',  fontsize=12)
axes[1].set_xlabel('Boron Family',  fontsize=12)
axes[2].set_xlabel('Halogen',  fontsize=12)

axes[0].set_ylabel('Count',  fontsize=13)


# We change the fontsize of minor ticks label 
axes[0].tick_params(axis='both', which='major', labelsize=12)
axes[1].tick_params(axis='both', which='minor', labelsize=16)

axes[1].tick_params(axis='both', which='major', labelsize=12)
axes[2].tick_params(axis='both', which='major', labelsize=12)

plt.subplots_adjust(wspace=0.05)
plt.legend(labels = ['direct bandgap', 'indirect bandgap'],
           bbox_to_anchor =(0.25, 1.20),
           frameon=False, labelspacing=1, fontsize=12, ncol=2)

In [ ]:
# Heatmap formation energies
escalax = ['Be','Mg','Ca','Sr','Ba']
escalay = ['B','Al','Ga','In','Tl']

heatmap_fenergy_F   = np.zeros((len(escalay),len(escalax)), dtype=np.float)
heatmap_fenergy_Cl  = np.zeros((len(escalay),len(escalax)), dtype=np.float)
heatmap_fenergy_Br  = np.zeros((len(escalay),len(escalax)), dtype=np.float)
heatmap_fenergy_I   = np.zeros((len(escalay),len(escalax)), dtype=np.float)

heatmap_direct_F = np.zeros((len(escalay),len(escalax)), dtype=object)
heatmap_direct_Cl = np.zeros((len(escalay),len(escalax)), dtype=object)
heatmap_direct_Br = np.zeros((len(escalay),len(escalax)), dtype=object)
heatmap_direct_I = np.zeros((len(escalay),len(escalax)), dtype=object)

    
for i in range(0, len(escalax)):
    for j in range(0, len(escalay)):
        dfx = results_df[results_df['A']==escalax[i]]
        dfy = dfx[dfx['B']==escalay[j]]
        
        heatmap_fenergy_F[i][j] = dfy[dfy['C']=='F']['bandgap'].values[0]
        heatmap_fenergy_Cl[i][j] = dfy[dfy['C']=='Cl']['bandgap'].values[0]
        heatmap_fenergy_Br[i][j] = dfy[dfy['C']=='Br']['bandgap'].values[0]
        heatmap_fenergy_I[i][j] = dfy[dfy['C']=='I']['bandgap'].values[0]
        
        #print(str(round(heatmap_fenergy_F[i][j],2))+'*')
        
        if not(dfy[dfy['C']=='F']['bandgap_direct'].values[0]):
            if dfy[dfy['C']=='F']['bandgap'].values[0] == 0:
                heatmap_direct_F[i][j] = ''
            else:
                heatmap_direct_F[i][j] = '*'
        else:
            heatmap_direct_F[i][j] = ''
            
        if not(dfy[dfy['C']=='Cl']['bandgap_direct'].values[0]):
            if dfy[dfy['C']=='Cl']['bandgap'].values[0] == 0:
                heatmap_direct_Cl[i][j] = ''
            else:
                heatmap_direct_Cl[i][j] = '*'
        else:
            heatmap_direct_Cl[i][j] = ''
            
        if not(dfy[dfy['C']=='Br']['bandgap_direct'].values[0]):
            if dfy[dfy['C']=='Br']['bandgap'].values[0] == 0:
                heatmap_direct_Br[i][j] = ''
            else:
                heatmap_direct_Br[i][j] = '*'
        else:
            heatmap_direct_Br[i][j] = ''
            
        if not(dfy[dfy['C']=='I']['bandgap_direct'].values[0]): 
            if dfy[dfy['C']=='I']['bandgap'].values[0] == 0:
                heatmap_direct_I[i][j] = ''
            else:
                heatmap_direct_I[i][j] = '*'
        else:
            heatmap_direct_I[i][j] = ''

            
nomes = ['Fluorine', 'Chlorine', 'Bromine', 'Iodine']
matrizes = [heatmap_fenergy_F, heatmap_fenergy_Cl, heatmap_fenergy_Br, heatmap_fenergy_I]
directs = [heatmap_direct_F, heatmap_direct_Cl, heatmap_direct_Br, heatmap_direct_I]

maior=[]
menor=[]
for mat in matrizes:
    maior.append(np.amax(mat))
    menor.append(np.amin(mat))
    
vmin = min(menor)
vmax = max(maior)

fig, ax = plt.subplots(1,4, figsize=(16,4), dpi=100)
cbar_ax = fig.add_axes([.91,.2,.03,.6])

for i, axis in enumerate(ax):
    
    labels = (np.asarray(["{1:.2f}{0}".format(string, value)
                      for string, value in zip(directs[i].flatten(),
                                               matrizes[i].flatten())])
         ).reshape(len(escalax), len(escalay))
    
    sns.heatmap(matrizes[i], annot=labels, fmt="", 
                linewidths=0.0, ax=axis,
                xticklabels=escalay, yticklabels=escalax,
                cbar=True,
                vmin=vmin,
                vmax=vmax,
                cmap="YlGnBu", #BuPu
                robust=False,
                cbar_ax = cbar_ax,
                cbar_kws={'label': 'Bandgap energy (eV)'})
    axis.invert_yaxis()
    if i==0:
        axis.set_ylabel('Alkaline Earth Metal', fontsize=14)
    axis.set_xlabel('Boron Group', fontsize=14)
    axis.set_title(nomes[i], fontsize=14)

plt.rcParams['font.size'] = 12
#plt.savefig('bandgap1.png')